In [2]:
import os


os.environ["CARAVELS_CONFIG_FILE"] = "/home/rdpds/Documents/caravels/configfiles/H01.json"

In [3]:
from datetime import datetime

from pymongo import MongoClient, DESCENDING

from utils import utils


class BuildingRepository:
    def __init__(self):
        self.config = utils.get_config()
        self.server = str(self.config['storage']['local']['server'])
        self.port = str(self.config['storage']['local']['port'])
        self.IOTS_READING = self.config['storage']['local']['iots_reading']
        self.FORECAST = self.config['storage']['local']['forecast']
        self.TOTALPOWER = self.config['storage']['local']['totalpower']
        self.TOTALPOWERHOUR = self.config['storage']['local']['totalpowerhour']
        self.CONFIG_DB = self.config['storage']['local']['config']

    def save_config(self):
        try:
            conf = {"config": "config", "auto_answer": True}
            client = MongoClient(self.server + ':' + self.port)
            client[self.CONFIG_DB[0]][self.CONFIG_DB[1]].insert_one(conf)
        except Exception as e:
            print("An exception occurred ::", e)
        finally:
            client.close()

        if self.config['app']['monitoring']:
            print('\nConfig\n', conf)

    def get_historic_interval_iots(self, start, end):
        client = MongoClient(self.server + ':' + self.port)
        historic = list(
            client[self.IOTS_READING[0]][self.IOTS_READING[1]].find({'datetime': {'$gt': start, '$lt': end}}))
        client.close()
        return historic

    def get_historic_iots(self, start):
        if type(start) is str:
            date_start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
        else:
            date_start = start
        client = MongoClient(self.server + ':' + self.port)
        historic = list(client[self.IOTS_READING[0]][self.IOTS_READING[1]].find(
            {'datetime': {'$gt': date_start}}))
        client.close()
        return historic

    def get_historic_hour(self, start):
        if type(start) is str:
            date_start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
        else:
            date_start = start
        client = MongoClient(self.server + ':' + self.port)
        historic = list(client[self.TOTALPOWERHOUR[0]][self.TOTALPOWERHOUR[1]].find({'datetime': {'$gt': date_start}}))
        client.close()
        return historic
    
    def get_historic_hour_interval(self, start: datetime, end: datetime):
        client = MongoClient(self.server + ':' + self.port)
        historic = list(client[self.TOTALPOWERHOUR[0]][self.TOTALPOWERHOUR[1]].find({'datetime': {'$gt': start, '$lt': end}}))
        client.close()
        return historic

    def iots_historic_update(self, start):
        client = MongoClient(self.server + ':' + self.port)
        historic = list(client[self.IOTS_READING[0]][self.IOTS_READING[1]].find({'datetime': {'$gt': start}}))
        client.close()
        client = MongoClient(self.server + ':' + self.port)
        for entry in historic:
            new_date = datetime.strptime(entry['datetime'], "%Y-%m-%d %H:%M:%S.%f")
            client[self.IOTS_READING[0]][self.IOTS_READING[1]].update_one({'datetime': entry['datetime']},
                                                                          {'$set': {'datetime': new_date}})

        client.close()
        return historic

    def get_iots_reading_col(self, time, time_emb):
        client = MongoClient(self.server + ':' + self.port)
        building_iot_reading = list(
            client[self.IOTS_READING[0]][self.IOTS_READING[1]].find({'datetime': {'$gt': time, '$lt': time_emb}}))
        client.close()
        return building_iot_reading

    def get_forecastvalue_col(self):
        client = MongoClient(self.server + ':' + self.port)
        forecastvalue = list(client.ForecastDay.forecastvalue.find().sort("_id", DESCENDING).limit(1))
        client.close()
        return forecastvalue

    def get_power_historic(self, start):
        client = MongoClient(self.server + ':' + self.port)
        building_totalpower = list(client[self.TOTALPOWER[0]][self.TOTALPOWER[1]].find({'datetime': {'$gt': start}}))
        client.close()
        return building_totalpower

    def get_power_historic_interval(self, start: datetime, end: datetime):
        client = MongoClient(self.server + ':' + self.port)
        building_totalpower = list(
            client[self.TOTALPOWER[0]][self.TOTALPOWER[1]].find({'datetime': {'$gt': start, '$lt': end}}))
        client.close()
        return building_totalpower

    def get_totalpower_col(self):
        client = MongoClient(self.server + ':' + self.port)
        building_totalpower = list(
            client[self.TOTALPOWER[0]][self.TOTALPOWER[1]].find().sort("datetime", -1).limit(100000))
        client.close()
        return building_totalpower

    def insert_total(self, totalpower, totalgeneration, datetime):
        try:
            total = {"totalpower": totalpower, "totalgeneration": totalgeneration, "datetime": datetime}
            client = MongoClient(self.server + ':' + self.port)
            client[self.TOTALPOWER[0]][self.TOTALPOWER[1]].insert_one(total)
        except Exception as e:
            print("An exception occurred ::", e)
        finally:
            client.close()

        if self.config['app']['monitoring']:
            print('\nTotal\n', total)

    def insert_forecast(self, forecast_power, datetime):
        try:
            forecast = {"forecast_power": forecast_power, "datetime": datetime}
            client = MongoClient(self.server + ':' + self.port)
            client[self.FORECAST[0]][self.FORECAST[1]].insert_one(forecast)
        except Exception as e:
            print("An exception occurred ::", e)
        finally:
            client.close()

        if self.config['app']['monitoring']:
            print('\nForecast\n', forecast)

    def insert_forecastday(self, iat, datetime):
        try:
            forecastday = {"forecast_power": {"0": str(iat[0, 0]),
                                              "1": str(iat[1, 0]),
                                              "2": str(iat[2, 0]),
                                              "3": str(iat[3, 0]),
                                              "4": str(iat[4, 0]),
                                              "5": str(iat[5, 0]),
                                              "6": str(iat[6, 0]),
                                              "7": str(iat[7, 0]),
                                              "8": str(iat[8, 0]),
                                              "9": str(iat[9, 0]),
                                              "10": str(iat[10, 0]),
                                              "11": str(iat[11, 0]),
                                              "12": str(iat[12, 0]),
                                              "13": str(iat[13, 0]),
                                              "14": str(iat[14, 0]),
                                              "15": str(iat[15, 0]),
                                              "16": str(iat[16, 0]),
                                              "17": str(iat[17, 0]),
                                              "18": str(iat[18, 0]),
                                              "19": str(iat[19, 0]),
                                              "20": str(iat[20, 0]),
                                              "21": str(iat[21, 0]),
                                              "22": str(iat[22, 0]),
                                              "23": str(iat[23, 0])},
                           "datetime": datetime}
            client = MongoClient(self.server + ':' + self.port)
            client.ForecastDay.forecastvalue.insert_one(forecastday)
        except Exception as e:
            print("An exception occurred ::", e)
        finally:
            client.close()

        if self.config['app']['monitoring']:
            print('\nForecastDay\n', forecastday)

    def insert_hour(self, start_hour, consumption, generation, flexibility):
        try:
            total = {"datetime": start_hour, "consumption": consumption, "generation": generation,
                     "flexibility": flexibility}
            client = MongoClient(self.server + ':' + self.port)
            client[self.TOTALPOWERHOUR[0]][self.TOTALPOWERHOUR[1]].insert_one(total)
        except Exception as e:
            print("An exception occurred ::", e)
        finally:
            client.close()

        if self.config['app']['monitoring']:
            print('\nTotal\n', total)


In [4]:
building_repo = BuildingRepository()

In [5]:
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
def get_historic_last_day_by_hour():
    print(datetime.strftime(datetime.now() - timedelta(hours=24), "%Y-%m-%d %H:%M:%S"))
    total = building_repo.get_historic_hour(
        datetime.strftime(datetime.now() - timedelta(hours=24), "%Y-%m-%d %H:%M:%S"))
    if len(total)==0:
        return []
    total = pd.DataFrame(total)
    total = total.drop(["_id"], axis=1)
    total = total.values.tolist()
    return total
get_historic_last_day_by_hour()

In [ ]:
def get_historic(start):
    total = building_repo.get_power_historic(start)
    if len(total)==0:
        return []
    total = pd.DataFrame(total)
    total = total.drop(["_id"], axis=1)
    return total.values.tolist()
get_historic(datetime.now().replace(minute=0, second=0, microsecond=0))

In [ ]:
import random
def get_historic_overview():
    historic = get_historic_last_day_by_hour()
    last_hour = get_historic(datetime.now().replace(minute=0, second=0, microsecond=0))
    total = pd.DataFrame(last_hour, columns=['consumption', 'generation', 'datetime'])
    total['datetime'] = pd.to_datetime(total['datetime'], format='%Y-%m-%d %H:%M:%S', dayfirst=True)
    total.set_index("datetime", inplace=True)
    total = total.resample('1h').mean()
    total = total.values.tolist()[0]
    historic.insert(0, [datetime.timestamp(datetime.now().replace(minute=0, second=0, microsecond=0)), total[0], total[1],
                        total[0] * random.randrange(5, 20) / 100])
    return historic
get_historic_overview()

In [9]:
def forecast_consumption():
    building_repo = BuildingRepository()
    now = datetime.now()
    start = now - timedelta(days=7, hours=now.hour, minutes=now.minute)
    building_totalpower = building_repo.get_historic_hour_interval(start, start + timedelta(days=1))
    consumption = []
    for line in building_totalpower:
        consumption.append([line['datetime'],line['consumption']])

    total = pd.DataFrame(consumption, columns=['datetime', 'consumption'])
    total['datetime'] = pd.to_datetime(total['datetime'], format='%Y-%m-%d %H:%M:%S', dayfirst=True)
    total.set_index("datetime", inplace=True)
    total = total.resample('1h').mean()
    return total['consumption'].values.tolist()

In [ ]:
from flask import jsonify
def overview():
    historic_overview = get_historic_overview()
    forecast = forecast_consumption()
    return {'historic': historic_overview, 'forecast': forecast}
overview()

In [13]:
import os
import json

with open("config/f.json") as file:
    data = json.load(file)

os.environ['CARAVELS_CONFIG'] = json.dumps(data)
data

{'app': {'port': 5000,
  'monitoring': 0,
  'dr_events_auto_accept': 1,
  'hour_offset': 0},
 'storage': {'_comment': 'database configuration using mongodb',
  'storing_frequency': 5,
  'local': {'type': 'mongodb',
   'server': '192.168.2.91',
   'port': 32017,
   'database': 'H01',
   'batteries': ['H01', 'batteries'],
   'iots_reading': ['H01', 'iots_reading'],
   'forecast': ['H01', 'forecast'],
   'totalpower': ['H01', 'totalpower'],
   'totalpowerhour': ['H01', 'totalpowerhour'],
   'token': ['H01', 'tokens'],
   'demand_response': ['H01', 'demand_response_invitations'],
   'config': ['H01', 'config'],
   'balance': ['H01', 'balance'],
   'benefit': ['H01', 'benefit'],
   'divisions': ['H01', 'divisions'],
   'p2p_prices': ['H01', 'p2p_prices'],
   'p2p_transaction': ['H01', 'p2p_transaction']}},
 'resources': {'_comment': 'reading period in seconds, resource response',
  'monitoring_period': 1,
  'batteries': [],
  'iots': [{'name': 'AC1',
    'type': 'hvac',
    'uri': 'http://1

In [20]:
from database.BuildingRepository import BuildingRepository
from database.P2PRepository import P2PRepository
from datetime import datetime, timedelta
import pandas as pd

def get_p2p_intervals():
    today = datetime.now().replace(hour=0,minute=0, second=0, microsecond=0) - timedelta(seconds=1)
    last_day = today - timedelta(days=1)
    repo = P2PRepository()
    p2p_transactions = repo.get_transactions(last_day, today)
    p2p_df = pd.DataFrame(p2p_transactions, columns=['datetime', 'quantity', 'cost'])
    p2p_df['datetime'] = pd.to_datetime(p2p_df['datetime'], format='%Y-%m-%dT%H:%M:%S.%fZ', dayfirst=True)
    p2p_df.set_index("datetime", inplace=True)
    p2p_df = p2p_df.resample('1h').mean()
    return {row['datetime']: row for row in p2p_df.reset_index().to_dict(orient='records')}

def get_intervals():
    today = datetime.now().replace(hour=0,minute=0, second=0, microsecond=0) - timedelta(seconds=1)
    last_day = today - timedelta(days=1)
    repo = BuildingRepository()
    totalpower = repo.get_power_historic_interval(last_day, today)
    total = pd.DataFrame(totalpower, columns=['totalpower', 'totalgeneration', 'datetime'])
    total['datetime'] = pd.to_datetime(total['datetime'], format='%Y-%m-%d %H:%M:%S', dayfirst=True)
    total.set_index("datetime", inplace=True)
    total = total.resample('1h').mean()
    total['datetime'] = total.index
    return total.to_dict(orient='records')


In [25]:
def get_energy_from_retailer():
        total = get_intervals()
        p2p_dict = get_p2p_intervals()
        print(p2p_dict)
        retailer = []
        for interval in total:
            datetime_interval = interval['datetime']
            p2p_sold = 0
            if datetime_interval in p2p_dict:
                p2p_entry = p2p_dict[datetime_interval]
                p2p_sold = p2p_entry['quantity']
                    
            retailer.append({"datetime": interval['datetime'], "retailer": interval['totalpower'] + p2p_sold - interval['totalgeneration']})
        #retailer = consumption + p2p_sold - generation - p2p_bought
        #retailer = consumption + (p2p_sold - p2p_bought) - generation 
        return retailer
k = get_energy_from_retailer()
k

{Timestamp('2024-12-19 12:00:00'): {'datetime': Timestamp('2024-12-19 12:00:00'), 'quantity': -12.0, 'cost': 12.0}}


[{'datetime': Timestamp('2024-12-19 00:00:00'),
  'retailer': 1376.2513888888889},
 {'datetime': Timestamp('2024-12-19 01:00:00'), 'retailer': 2003.036111111111},
 {'datetime': Timestamp('2024-12-19 02:00:00'), 'retailer': 3844.626388888889},
 {'datetime': Timestamp('2024-12-19 03:00:00'), 'retailer': 4199.4125},
 {'datetime': Timestamp('2024-12-19 04:00:00'),
  'retailer': 3784.3069444444445},
 {'datetime': Timestamp('2024-12-19 05:00:00'),
  'retailer': 1990.7888888888888},
 {'datetime': Timestamp('2024-12-19 06:00:00'),
  'retailer': 1272.7569444444443},
 {'datetime': Timestamp('2024-12-19 07:00:00'), 'retailer': 592.7236111111112},
 {'datetime': Timestamp('2024-12-19 08:00:00'),
  'retailer': 501.98333333333335},
 {'datetime': Timestamp('2024-12-19 09:00:00'), 'retailer': 254.8291666666667},
 {'datetime': Timestamp('2024-12-19 10:00:00'),
  'retailer': -328.9291666666667},
 {'datetime': Timestamp('2024-12-19 11:00:00'),
  'retailer': -452.2972222222222},
 {'datetime': Timestamp('20

In [32]:
def get_self_consumption():
    total = get_intervals()
    p2p_dict = get_p2p_intervals()
    self_consumption = []
    for interval in total:
        datetime_interval = interval['datetime']
        p2p_sold = 0
        if datetime_interval in p2p_dict:
            p2p_entry = p2p_dict[datetime_interval]
            if p2p_entry['quantity'] < 0 and p2p_entry['cost'] > 0:  # P2P Sold
                p2p_sold = abs(p2p_entry['quantity'])

        if interval['totalgeneration'] - p2p_sold < interval['totalpower']:
            self_consumption.append({"datetime": interval['datetime'], "self_consumption": interval['totalgeneration'] - p2p_sold})
        else:
            self_consumption.append({"datetime": interval['datetime'], "self_consumption": interval['totalpower']})
    return self_consumption
k = get_self_consumption()

In [38]:
co2_constant = 20
def get_co2_with_p2p():
    retailer = get_energy_from_retailer()
    co2 = []
    for interval in retailer:  
        if interval['retailer'] > 0:
            co2.append({"datetime": interval['datetime'], "co2_with_p2p": round(interval['retailer']/1000 * co2_constant,2)})
        else:
            co2.append({"datetime": interval['datetime'], "co2_with_p2p": 0})
    return co2

get_co2_with_p2p()

{Timestamp('2024-12-19 12:00:00'): {'datetime': Timestamp('2024-12-19 12:00:00'), 'quantity': -12.0, 'cost': 12.0}, Timestamp('2024-12-19 13:00:00'): {'datetime': Timestamp('2024-12-19 13:00:00'), 'quantity': 12.0, 'cost': -12.0}}


[{'datetime': Timestamp('2024-12-19 00:00:00'), 'co2_with_p2p': 27.53},
 {'datetime': Timestamp('2024-12-19 01:00:00'), 'co2_with_p2p': 40.06},
 {'datetime': Timestamp('2024-12-19 02:00:00'), 'co2_with_p2p': 76.89},
 {'datetime': Timestamp('2024-12-19 03:00:00'), 'co2_with_p2p': 83.99},
 {'datetime': Timestamp('2024-12-19 04:00:00'), 'co2_with_p2p': 75.69},
 {'datetime': Timestamp('2024-12-19 05:00:00'), 'co2_with_p2p': 39.82},
 {'datetime': Timestamp('2024-12-19 06:00:00'), 'co2_with_p2p': 25.46},
 {'datetime': Timestamp('2024-12-19 07:00:00'), 'co2_with_p2p': 11.85},
 {'datetime': Timestamp('2024-12-19 08:00:00'), 'co2_with_p2p': 10.04},
 {'datetime': Timestamp('2024-12-19 09:00:00'), 'co2_with_p2p': 5.1},
 {'datetime': Timestamp('2024-12-19 10:00:00'), 'co2_with_p2p': 0},
 {'datetime': Timestamp('2024-12-19 11:00:00'), 'co2_with_p2p': 0},
 {'datetime': Timestamp('2024-12-19 12:00:00'), 'co2_with_p2p': 0},
 {'datetime': Timestamp('2024-12-19 13:00:00'), 'co2_with_p2p': 0},
 {'datetim

In [39]:
def get_co2_without_p2p():
    total = get_intervals()
    co2 = []
    for interval in total:
        if interval['totalpower']- interval['totalgeneration'] > 0:
            co2.append({"datetime": interval['datetime'], "co2_without_p2p": round((interval['totalpower']- interval['totalgeneration'])/1000 * co2_constant,2)})
        else:
            co2.append({"datetime": interval['datetime'], "co2_without_p2p": 0})
    return co2
get_co2_without_p2p()

[{'datetime': Timestamp('2024-12-19 00:00:00'), 'co2_without_p2p': 27.53},
 {'datetime': Timestamp('2024-12-19 01:00:00'), 'co2_without_p2p': 40.06},
 {'datetime': Timestamp('2024-12-19 02:00:00'), 'co2_without_p2p': 76.89},
 {'datetime': Timestamp('2024-12-19 03:00:00'), 'co2_without_p2p': 83.99},
 {'datetime': Timestamp('2024-12-19 04:00:00'), 'co2_without_p2p': 75.69},
 {'datetime': Timestamp('2024-12-19 05:00:00'), 'co2_without_p2p': 39.82},
 {'datetime': Timestamp('2024-12-19 06:00:00'), 'co2_without_p2p': 25.46},
 {'datetime': Timestamp('2024-12-19 07:00:00'), 'co2_without_p2p': 11.85},
 {'datetime': Timestamp('2024-12-19 08:00:00'), 'co2_without_p2p': 10.04},
 {'datetime': Timestamp('2024-12-19 09:00:00'), 'co2_without_p2p': 5.1},
 {'datetime': Timestamp('2024-12-19 10:00:00'), 'co2_without_p2p': 0},
 {'datetime': Timestamp('2024-12-19 11:00:00'), 'co2_without_p2p': 0},
 {'datetime': Timestamp('2024-12-19 12:00:00'), 'co2_without_p2p': 0},
 {'datetime': Timestamp('2024-12-19 13:

In [6]:
from datetime import datetime, timedelta
end = datetime.now().replace(day=1,hour=0,minute=0, second=0, microsecond=0)
start = end - timedelta(days=2)
start = start.replace(day=1)


(datetime.datetime(2024, 12, 1, 0, 0), datetime.datetime(2025, 1, 1, 0, 0))